In [2]:
import bs4 as bs
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import os
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests
import numpy as np

#Als erstes sollen den einzelnen Produktionszahlen nach Modellen mit den Modellpreisen verrechnet werden:

#Import der Tabelle welche die Preisinformationen enthält:

df = pd.read_excel('Modell - Stadt - Typ - Preise' + '.xlsx', 'Tabelle1')

#Import der monatlichen Flugzeugproduktionszahlen

df1 = pd.read_excel('FlugzeugdatenVersion1' + '.xlsx', 'Tabelle1')

#erstellen eines Index:

dates = pd.date_range('1929-12-31','1946-01-01', freq='M')
df1.index = dates
df1 = df1.drop('Zeit', 1)

#kumulieren der  Produktionszahlen und resamplen, so dass aus den monatlichen Zahlen jährliche werden:
#(ich den Schritt durch geführt bevor ich entschieden habe mit nicht kumulierten Daten zu arbeiten, später wird die kumulation wieder aufgelöst)

df1 = df1.cumsum()
df1 = df1.resample('Y', label='right',).last()
df1 = df1.drop(pd.Timestamp('1929-12-31 00:00:00'))
df1 = df1.drop(df1.loc[df1.index > '1940-12-31 00:00:00'].index)



df.index = df['Flugzeugmodell']
del df.index.name

#neuer Datensatz, der nur die Preise beinhaltet

df2 = pd.DataFrame(df['Preis'])

#der Preisvektor wird transponiert und vervielfältigt, um im Anschluss die kumulierten Zahlen spaltenweise mit den Preisen zu multiplizieren

df2 = df2.transpose()
df2 = pd.concat([df2]*11)
dates1 = pd.date_range('1930-12-31','1941-01-01', freq='Y')
df2.index = dates1
df3 = df2.mul(df1)

#print(df.head())
#print(df3.head())

#df3.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Felix Fikowski Bachelor Thesis\airplane construction by price annual.xlsx', index = True)
#df.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Felix Fikowski Bachelor Thesis\Flugzeugdaten3.csv', index = True)






       Flugzeugmodell                                  Firma            Stadt  \
Bf 109         Bf 109            Bayerische Flugzeugwerke AG         Augsburg   
He 111         He 111       Ernst Heinkel Flugzeugwerke GmbH          Rostock   
Ju 87           Ju 87  Junkers Flugzeug- und Motorenwerke AG           Dessau   
Ju 52           Ju 52  Junkers Flugzeug- und Motorenwerke AG           Dessau   
Do 17           Do 17                     Dornier-Werke GmbH  Friedrichshafen   

                      Type  Preis  Unnamed: 5  \
Bf 109               Jäger   45.0        45.0   
He 111              Bomber  203.9       203.9   
Ju 87   Sturzkampfflugzeug  100.3       100.3   
Ju 52            Verbinder  125.8       125.8   
Do 17               Bomber  185.5       185.5   

                                               Unnamed: 6  
Bf 109  Bundesarchiv/Militärarchiv Freiburg, RL 3, Pro...  
He 111                                                NaN  
Ju 87                                 

In [28]:
#Im Folgenden werden die insgesamt Produzierten Zahlen nach Modellen den unterschiedlichen Gemeinden zugeordnet. 
#Daraus werden prozentuale Anteile der Gemeinden je Modell gebildet:

#Import der Matrix, welchen die prodzuzierten Stückzahlen nach Modellen den Werkssitzen zuweist

df4 = pd.read_excel('aircraft manufactorer network matrix' + '.xlsx', 'Tabelle1')

#Gruppierung nach Stadt, da manche Modellen von unterschiedlichen Firmen in der gleichen Stadt gebaut wurden:

df4 = df4.groupby('Stadt').sum()
del df4.index.name
df4 = df4.drop('Summe:', 0)

#Alphabetische Sortierung nach Städten:

df4 = df4.sort_index()

#Aus den absoluten Zahlen werden prozentuale Anteile an der Gesamtproduktion nach Modell errechnet:

df4 = df4/df4[df4.columns].sum()
df4 = df4.fillna(0)


print(df4.head())

#df4.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Felix Fikowski Bachelor Thesis\aircraft manufactorer network matrix percentage.xlsx', index = True)


                            Bf 109    He 111     Ju 87  Ju 52  Do 17  \
Augsburg                  0.048271  0.000000  0.000000    0.0    0.0   
Berlin                    0.000000  0.000000  0.000000    0.0    0.0   
Brandenburg an der Havel  0.000000  0.142948  0.000000    0.0    0.0   
Braunschweig              0.000000  0.000000  0.000000    0.0    0.0   
Bremen                    0.046314  0.000000  0.629556    0.0    0.0   

                            Bf 110  He 112  He 115  Hs 123  Bü 131  ...  \
Augsburg                  0.334728     0.0     0.0     0.0     0.0  ...   
Berlin                    0.000000     0.0     0.0     0.0     1.0  ...   
Brandenburg an der Havel  0.000000     0.0     0.0     0.0     0.0  ...   
Braunschweig              0.189679     0.0     0.0     0.0     0.0  ...   
Bremen                    0.226639     0.0     0.0     0.0     0.0  ...   

                          BV 138  He 177  Bü 181  Ar 65  Kl 25  Fw 56  \
Augsburg                     0.0     0.0   

In [29]:
#Nun wird der Produktionswert je Modell je Jahr den verschiedenen Gemeinden zugeordnet.

df3 =df3.astype(float)

#Transponieren der Daten, die den Gemeinden die prozentuallen Anteile an den Modellen zuweist:

df4 = df4.T

#Anpasssen der Tabellen Beschriftung für die multiplikation beider Matrizen:

df3.columns.name = 'Modell'
df3.index.name = 'Zeit'
df4.columns.name = 'Stadt'
df4.index.name = 'Modell'

#Matrizen Multiplikation:

df5 = np.dot(df3, df4)
df5 = pd.DataFrame(df5)

#Index und Spaltennamen setzen beim neuen Datensatz:

df5.index = df3.index
df5.columns = df4.columns

print(df5)

#df5.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Felix Fikowski Bachelor Thesis\cum aircraftprices by city.xlsx', index = True)


Stadt            Augsburg        Berlin  Brandenburg an der Havel  \
Zeit                                                                
1930-12-31       0.000000      0.000000                  0.000000   
1931-12-31       0.000000      0.000000                  0.000000   
1932-12-31     469.097468      0.000000                 12.980769   
1933-12-31    4481.670377     12.177267                311.538462   
1934-12-31    9747.217039    292.886965               3639.485298   
1935-12-31   13600.901084   1736.813287              12928.448679   
1936-12-31   16129.134918   5319.777393              31696.454286   
1937-12-31   20633.234435   8165.968569              46371.102268   
1938-12-31   28902.047366  11244.998756              66965.919613   
1939-12-31   52067.796061  21111.956321             112377.546022   
1940-12-31  109977.030769  98973.334404             203476.817949   

Stadt        Braunschweig         Bremen         Dessau  Friedrichshafen  \
Zeit                      

In [30]:
#runden der Daten auf zwei Stellen und Selektieren der Jahre 1931 - 1936:

df5 = df5.round(2)
df5 = df5.iloc[ 1:7 , : ].T
print(df5)
df5.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Felix Fikowski Bachelor Thesis\Flugzeugproduktion.xlsx', index = True)

Zeit                      1931-12-31  1932-12-31  1933-12-31  1934-12-31  \
Stadt                                                                      
Augsburg                        0.00      469.10     4481.67     9747.22   
Berlin                          0.00        0.00       12.18      292.89   
Brandenburg an der Havel        0.00       12.98      311.54     3639.49   
Braunschweig                   14.82       42.59     1585.17     7773.47   
Bremen                         18.29      706.81     6646.95    16329.23   
Dessau                          0.00     1077.98     2910.56    21244.08   
Friedrichshafen                 0.00        0.00      423.07     1357.04   
Gotha                           4.28      252.14     2849.84     6644.39   
Halle                          23.03       46.06     2054.43     8013.64   
Hamburg                         0.00        0.00        0.00        0.00   
Kassel                          2.14        4.28      203.95     3357.38   
Leipzig     

In [31]:
#Im folgenden wird die Kumulierung wieder aufgehoben:

yearly = pd.read_excel('Flugzeugproduktion' + '.xlsx', 'Sheet1')

#Zunächst werden die Indizes richtig gesetzt und die Tabelle transformiert:

yearly.index = yearly['Stadt']
yearly = yearly.drop(['Stadt'], axis=1)
yearly = yearly.T
print(yearly.head())

Stadt                Augsburg   Berlin  Brandenburg an der Havel  \
1931-12-31 00:00:00      0.00     0.00                      0.00   
1932-12-31 00:00:00    469.10     0.00                     12.98   
1933-12-31 00:00:00   4481.67    12.18                    311.54   
1934-12-31 00:00:00   9747.22   292.89                   3639.49   
1935-12-31 00:00:00  13600.90  1736.81                  12928.45   

Stadt                Braunschweig    Bremen    Dessau  Friedrichshafen  \
1931-12-31 00:00:00         14.82     18.29      0.00             0.00   
1932-12-31 00:00:00         42.59    706.81   1077.98             0.00   
1933-12-31 00:00:00       1585.17   6646.95   2910.56           423.07   
1934-12-31 00:00:00       7773.47  16329.23  21244.08          1357.04   
1935-12-31 00:00:00      14155.33  29622.63  58404.23         38131.38   

Stadt                  Gotha     Halle  Hamburg   Kassel   Leipzig  München  \
1931-12-31 00:00:00     4.28     23.03     0.00     2.14      0.00 

In [32]:
#Auflösen der Kumulation und Auswahl der Daten ab 1932

yearly = yearly.diff().fillna(yearly)
yearly = yearly.iloc[ 1:6 , : ]

print(yearly.head(6))
yearly.to_excel(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Bilder\Flugzeugproduktionjährlich.xlsx', index = True)

Stadt                Augsburg   Berlin  Brandenburg an der Havel  \
1932-12-31 00:00:00    469.10     0.00                     12.98   
1933-12-31 00:00:00   4012.57    12.18                    298.56   
1934-12-31 00:00:00   5265.55   280.71                   3327.95   
1935-12-31 00:00:00   3853.68  1443.92                   9288.96   
1936-12-31 00:00:00   2528.23  3582.97                  18768.00   

Stadt                Braunschweig    Bremen    Dessau  Friedrichshafen  \
1932-12-31 00:00:00         27.77    688.52   1077.98             0.00   
1933-12-31 00:00:00       1542.58   5940.14   1832.58           423.07   
1934-12-31 00:00:00       6188.30   9682.28  18333.52           933.97   
1935-12-31 00:00:00       6381.86  13293.40  37160.15         36774.34   
1936-12-31 00:00:00       8432.98  12923.76  46550.40         29221.79   

Stadt                  Gotha    Halle  Hamburg    Kassel   Leipzig  München  \
1932-12-31 00:00:00   247.86    23.03     0.00      2.14    185.61 

In [40]:
#Hinzufügen einer Summenzeile zur späteren Grafikerstellung.

print(type(yearly))
yearly = yearly.T
yearly.loc["Summe"] = yearly.sum()
print(yearly.tail())

<class 'pandas.core.frame.DataFrame'>
               1932      1933      1934       1935       1936
Stadt                                                        
Regensburg     0.00      0.00    123.75     247.49    1812.80
Rostock     1056.97   7098.69  20833.23   43199.63   49008.94
Stuttgart      0.00   1694.60   3190.73     626.23     174.66
Wismar         0.00      0.00    130.68       0.00    1088.95
Summe       3877.40  28240.56  86612.14  176161.76  227086.81


In [41]:
yearly.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Bilder\Flugzeugproduktionjährlich.csv', index = True)

In [49]:
df5 = pd.read_excel('Flugzeugproduktion' + '.xlsx', 'Sheet1')
df5.to_csv(r'C:\Users\Felix\Documents\Studium\Bachelorarbeit Papadia\Bilder\Flugzeugproduktion.csv', index = True)

In [287]:
print(df.head())
print(df1.head())
print(df2.head())
print(df3.head())
print(df4.head())
print(df5.head())
print(df6.head())

       Flugzeugmodell                                  Firma            Stadt  \
Modell                                                                          
Bf 109         Bf 109            Bayerische Flugzeugwerke AG         Augsburg   
He 111         He 111       Ernst Heinkel Flugzeugwerke GmbH          Rostock   
Ju 87           Ju 87  Junkers Flugzeug- und Motorenwerke AG           Dessau   
Ju 52           Ju 52  Junkers Flugzeug- und Motorenwerke AG           Dessau   
Do 17           Do 17                     Dornier-Werke GmbH  Friedrichshafen   

                      Type  Preis  
Modell                             
Bf 109               Jäger   45.0  
He 111              Bomber  203.9  
Ju 87   Sturzkampfflugzeug  100.3  
Ju 52            Verbinder  125.8  
Do 17               Bomber  185.5  
            Bf 109  He 111  Ju 87  Ju 52  Do 17  Bf 110  He 112  He 115  \
1930-12-31       0       0      0      0      0       0       0       0   
1931-12-31       0       0    